In [ ]:
import torch
from torch import nn
class Discriminator(nn.Module):
  def __init__(self,channels_img,features_d):
    super(Discriminator,self).__init__()
    self.disc = nn.Sequential(
        # input : N x channels_img x 64 x 64
        nn.Conv2d(
            channels_img,
            features_d,
            kernel_size=4,
            stride=2,
            padding=1
        ),
        nn.LeakyReLU(0.2),
        self._block(features_d,features_d*2,4,2,1),# 16x16
        self._block(features_d*2,features_d*4,4,2,1), #  8x8
        self._block(features_d*4,features_d*8,4,2,1), # 4x4
        nn.Conv2d(features_d*8,1,kernel_size=4,stride=2,padding=0), # 1x1
        nn.Sigmoid()

    )
  def _block(self,in_channels,out_channels,kernel_size,stride,padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2)
    )
  def forward(self,x):
    return self.disc(x)
class Generator(nn.Module):
  def __init__(self,z_dim,channles_img,features_g):
    super(Generator,self).__init__()
    self.gen = nn.Sequential(
        # input: N x z_dim x 1x1
        self._block(z_dim,features_g*16,4,1,0), # N x f_g*16*4*4
        self._block(features_g*16,features_g*8,4,2,1), # 8x8
        self._block(features_g*8,features_g*4,4,2,1), # 16 x 16
        self._block(features_g*4,features_g*2,4,2,1), # 32x32
        nn.ConvTranspose2d(
            features_g*2,channles_img,kernel_size=4,stride=2,padding=1
        ),
        nn.Tanh()
    )
  def _block(self,in_channels,out_channles,kernel_size,stride,padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channles,
            kernel_size,
            stride,
            padding,
          bias=False),
        nn.LazyBatchNorm2d(out_channles),
        nn.ReLU()
    )
  def forward(self,x):
    return self.gen(x)


def initialize_weights(model):
  for m in model.modules():
    if isinstance(m,(nn.Conv2d,nn.ConvTranspose2d,nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data,0.0,0.02)
def test():
  N,in_channels,H,W = 0,3,64,64
  z_dim = 100
  x = torch.randn((N,in_channels,H,W))
  disc = Discriminator(in_channels,8)
  initialize_weights(disc)
  assert disc(x).shape == (N,1,1,1)
  gen = Generator(z_dim,in_channels,8)
  initialize_weights(gen)
  z = torch.randn(N,z_dim,1,1)
  assert gen(z).shape ==(N,in_channels,H,W)
test()

In [ ]:
!pip install TensorBoard==1.14


In [ ]:
import os
image_path = 'dawin_data'
image_path

# Get list of all files and directories in image_path
all_files = os.listdir(image_path)

# Loop through the list and print only directories
for file in all_files:
    if os.path.isdir(os.path.join(image_path, file)):
        print(file)
import zipfile
from pathlib import Path
with zipfile.ZipFile('twice/dawin_data.zip','r') as zip_ref:
  print('Unzipping data')
  zip_ref.extractall()

import pathlib
image_path = pathlib.Path(image_path)  


In [ ]:
# !rm -r dawin_data/train/tuzyu

In [ ]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir



In [ ]:
from torch import optim
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
import torch
from torch import nn
from dcgan import Discriminator, Generator, initialize_weights
# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
larning_rate = 2e-4
batch = 128
image_size =64
channles_img = 3
z_dim = 100
NOISE_DIM = 100
num_epoch = 5
features_disc = 64
features_gen = 64
transforms = transforms.Compose(
    [
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(channles_img)],[0.5 for _ in range(channles_img)]),
    ]
)
# dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms,download=True)
dataset = datasets.ImageFolder(root=train_dir,transform=transforms,target_transform=None)
loader = DataLoader(dataset,batch,shuffle=True)
gen = Generator(NOISE_DIM,channles_img,features_gen).to(device)
disc = Discriminator(channles_img,features_disc).to(device)
initialize_weights(gen)
initialize_weights(disc)
opt_gen = optim.Adam(gen.parameters(),lr=larning_rate,betas=(0.5,0.999))
opt_disc =optim.Adam(disc.parameters(),lr=larning_rate,betas=(0.5,0.999))
criterion = nn.BCELoss()
fixed_noise = torch.randn(32,z_dim,1,1).to(device)
step=0
gen.train()
disc.train()
for epoch in range(0,num_epoch):
  for batch_idx,(real,_)in enumerate(loader):
    real = real.to(device)
    noise = torch.randn((batch,z_dim,1,1)).to(device)
    fake = gen(noise)
    ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
    disc_real = disc(real).reshape(-1)
    loss_disc_real = criterion(disc_real,torch.zeros_like(disc_real))
    disc_fake = disc(fake).reshape(-1)
    loss_disc_fake = criterion(disc_fake,torch.zeros_like(disc_fake))
    loss_disc = (loss_disc_real+loss_disc_fake)/2
    disc.zero_grad()
    loss_disc.backward(retain_graph=True)
    opt_disc.step()
    ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
    output = disc(fake).reshape(-1)
    loss_gen = criterion(output,torch.ones_like(output))
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()
    if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epoch}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                img_grid_fake = torchvision.utils.make_grid(real, normalize=True)
                img_grid_real = torchvision.utils.make_grid(fake, normalize=True)

                # Print the fake and real images to tensorboard
                # writer_fake.add_image(
                #     "Mnist Fake Images", img_grid_fake, global_step=step
                # )
                # writer_real.add_image(
                #     "Mnist Real Images", img_grid_real, global_step=step
                # )
                step += 1


In [ ]:
"""
Training of DCGAN network on MNIST dataset with Discriminator
and Generator imported from models.py
Programmed by Aladdin Persson <aladdin.persson at hotmail dot com>
* 2020-11-01: Initial coding
* 2022-12-20: Small revision of code, checked that it works with latest PyTorch version
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# from torch.utils.tensorboard import SummaryWriter


# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for disc
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
NOISE_DIM = 100
NUM_EPOCHS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64

transforms = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)


# If you train on MNIST, remember to set channels_img to 1
dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms, download=True
)

# comment mnist above and uncomment below if train on CelebA
# dataset = datasets.ImageFolder(root="celeb_dataset", transform=transforms)
# dataset = datasets.ImageFolder(root=train_dir,transform=transforms)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, NOISE_DIM, 1, 1).to(device)
# writer_real = SummaryWriter(f"logs/real")
# writer_fake = SummaryWriter(f"logs/fake")
step = 0

gen.train()
disc.train()

for epoch in range(NUM_EPOCHS):
    # Target labels not needed! <3 unsupervised
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

                # writer_real.add_image("Real", img_grid_real, global_step=step)
                # writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1

In [ ]:
# after the training loop
torch.save(gen.state_dict(), "dahwin_dahyun.pth")

In [ ]:
# create a new instance of the generator
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN ).to(device)


# load the saved state of the generator
gen.load_state_dict(torch.load("dahwin_dahyun.pth"))

In [ ]:
import torch
import torchvision
from torchvision.utils import save_image

# Define the hyperparameters
NOISE_DIM = 100
channles_img = 1
features_gen = 64
image_size = 64

# Create a new instance of the generator
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN ).to(device)

# Load the saved state of the generator
gen.load_state_dict(torch.load("dahwin_dahyun.pth"))

# Generate 64 images
num_images = 64
noise = torch.randn(num_images, NOISE_DIM, 1, 1).to(device)
generated_images = gen(noise)

# Save the generated images to a file
save_image(generated_images, "generated_images.png", nrow=8, normalize=True)

# Display the generated images
from PIL import Image
im = Image.open("generated_images.png")
im.show()


In [ ]:
# create a new instance of the generator
gen = Generator(NOISE_DIM, channles_img, features_gen).to(device)

# load the saved state of the generator
gen.load_state_dict(torch.load("dahwin_dahyun.pth"))

# generate a batch of random noise vectors
num_images = 32
noise = torch.randn(num_images, NOISE_DIM, 1, 1).to(device)

# generate fake images from the random noise
with torch.no_grad():
    fake = gen(noise)

# visualize the generated images
import matplotlib.pyplot as plt
import numpy as np
img_grid = torchvision.utils.make_grid(fake, normalize=True)
plt.imshow(np.transpose(img_grid.cpu().numpy(), (1,2,0)))
plt.axis('off')
plt.show()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs


In [ ]:
%tensorboard --logdir logs